# Sampling multiplier 
*Important* The data in the sources here is too big to upload to GitHub, the links to the original sources are below, and will have to be placed inside this script if you wish to recalculate sampling multipliers using the same data.

We want a realistic sampling bias in our simulations. This sheet computes sampling multipliers for each country in our simulation by calculating the cases sequenced to number of cases ratio for each country, the most available dataset for this is COVID.

COVID sequencing data is obtained from https://www.biorxiv.org/content/10.1101/2024.07.12.603240v1
COVID case data is obtained from https://ourworldindata.org/covid-cases


In [8]:
import pandas as pd
import pycountry
unclean_seq_m = pd.read_csv('data_sources/Viridian_2M_noShortDel_MAPLE_metaData.tsv', sep='\t')
cases_by_country = pd.read_csv('data_sources/owid-covid-data.csv')

In [9]:
seq_m = unclean_seq_m[unclean_seq_m['country'].notna()]
seq_m = seq_m[seq_m['country']!= 'UNKNOWN']
seq_m.head()

,strain,collapsedTo,support,rootSupport,supportGroup,supportTo,mutationsInf,Ns,country,date,lineage
1,ERR4806745,ERR4806745_MinorSeqsClade,NaN,NaN,NaN,NaN,NaN,NaN,United Kingdom,2020-10-08,A
2,ERR5989690,ERR4806745_MinorSeqsClade,NaN,NaN,NaN,NaN,NaN,NaN,United Kingdom,2020,A
3,ERR6061709,ERR4806745_MinorSeqsClade,NaN,NaN,NaN,NaN,NaN,NaN,United Kingdom,2020,A
4,ERR5628278,ERR4806745_MinorSeqsClade,NaN,NaN,NaN,NaN,NaN,NaN,United Kingdom,2020,A
5,ERR4806964,ERR4806745_MinorSeqsClade,NaN,NaN,NaN,NaN,NaN,NaN,United Kingdom,2020-10-13,A


In [10]:
seq_count = seq_m.groupby('country').size()
def convert(country_name):
    country = pycountry.countries.get(name=country_name)
    if country is None:
        if country_name == 'Russia':
            return 'RUS'
        return country_name
    return country.alpha_3
seq_count.index = seq_count.index.map(lambda x: convert(x))
seq_count.head()

country
AGO      4
AUS     95
CAN    161
CHN     11
EST     13
dtype: int64

In [11]:
total_cases_by_country = cases_by_country.groupby('iso_code', as_index=True).aggregate({'new_cases':'sum'})
# Hong Kong is missing, using: https://www.worldometers.info/coronavirus/country/china-hong-kong-sar/

total_cases_by_country.loc['HKG', 'new_cases'] = 2937609.

In [12]:
sampling_proportions = pd.DataFrame(columns=['alpha2','proportion'])
new_index = 0
for country in seq_count.index:
    sampling_prop = seq_count[country]/total_cases_by_country.loc[country]['new_cases']
    sampling_proportions.loc[new_index] = [pycountry.countries.get(alpha_3=country).alpha_2]+[sampling_prop]
    new_index+=1
sampling_proportions['proportion'] = sampling_proportions['proportion'] 
sampling_proportions

,alpha2,proportion
0,AO,3.722004e-05
1,AU,8.009334e-06
2,CA,3.342462e-05
3,CN,1.107028e-07
4,EE,2.129503e-05
5,ET,3.990622e-06
6,FI,4.467524e-05
7,FR,2.666838e-06
8,DE,7.804826e-08
9,IN,3.108296e-07


In [13]:
sampling_proportions.to_csv('data_sources/sampling_multipliers.csv', index=False)

In [16]:
total_cases_by_country.loc['GBR']

new_cases    24956066.0
Name: GBR, dtype: float64